In [1]:
import numpy as np
np.random.seed(1)

# 2 im2col関数とcol2im関数

In [2]:
def im2col(x, fil_size, y_size, stride, pad):
    x_b, x_c, x_h, x_w = x.shape
    fil_h, fil_w = fil_size, fil_size
    y_h, y_w = y_size, y_size
    index = -1
    
    x_pad = np.pad(x, [(0, 0), (0, 0), (pad, pad), (pad, pad)], "constant")
    x_col = np.zeros((fil_h*fil_w, x_b, x_c, y_h, y_w))
    
    for h in range(fil_h):
        h2 = h + y_h*stride
        for w in range(fil_w):
            index += 1
            w2 = w + y_w*stride
            x_col[index,:,:,:,:] = x_pad[:,:,h:h2:stride,w:w2:stride]
    x_col = x_col.transpose(2,0,1,3,4).reshape(x_c*fil_h*fil_w, x_b*y_h*y_w)
    
    return x_col

def col2im(dx_col, x_shape, fil_size, y_size, stride, pad):
    x_b, x_c, x_h, x_w = x_shape
    fil_h, fil_w = fil_size, fil_size
    y_h, y_w = y_size, y_size
    index = -1
    
    dx_col = dx_col.reshape(x_c, fil_h*fil_w, x_b, y_h, y_w).transpose(1,2,0,3,4)
    dx = np.zeros((x_b, x_c, x_h+2*pad+stride-1, x_w+2*pad+stride-1))
    
    for h in range(fil_h):
        h2 = h + y_h*stride
        for w in range(fil_w):
            index += 1
            w2 = w + y_w*stride
            dx[:,:,h:h2:stride,w:w2:stride] += dx_col[index,:,:,:,:]
    
    return dx[:,:,pad:x_h+pad, pad:x_w+pad]

# 3 Pooling実装

## 3.1 順伝播

In [3]:
x = np.random.randint(0,10,2*3*4*4).reshape(2,3,4,4)
x

array([[[[5, 8, 9, 5],
         [0, 0, 1, 7],
         [6, 9, 2, 4],
         [5, 2, 4, 2]],

        [[4, 7, 7, 9],
         [1, 7, 0, 6],
         [9, 9, 7, 6],
         [9, 1, 0, 1]],

        [[8, 8, 3, 9],
         [8, 7, 3, 6],
         [5, 1, 9, 3],
         [4, 8, 1, 4]]],


       [[[0, 3, 9, 2],
         [0, 4, 9, 2],
         [7, 7, 9, 8],
         [6, 9, 3, 7]],

        [[7, 4, 5, 9],
         [3, 6, 8, 0],
         [2, 7, 7, 9],
         [7, 3, 0, 8]],

        [[7, 7, 1, 1],
         [3, 0, 8, 6],
         [4, 5, 6, 2],
         [5, 7, 8, 4]]]])

In [4]:
x_col = im2col(x,2,2,2,0).T.reshape(-1,4)
x_col

array([[5., 8., 0., 0.],
       [4., 7., 1., 7.],
       [8., 8., 8., 7.],
       [9., 5., 1., 7.],
       [7., 9., 0., 6.],
       [3., 9., 3., 6.],
       [6., 9., 5., 2.],
       [9., 9., 9., 1.],
       [5., 1., 4., 8.],
       [2., 4., 4., 2.],
       [7., 6., 0., 1.],
       [9., 3., 1., 4.],
       [0., 3., 0., 4.],
       [7., 4., 3., 6.],
       [7., 7., 3., 0.],
       [9., 2., 9., 2.],
       [5., 9., 8., 0.],
       [1., 1., 8., 6.],
       [7., 7., 6., 9.],
       [2., 7., 7., 3.],
       [4., 5., 5., 7.],
       [9., 8., 3., 7.],
       [7., 9., 0., 8.],
       [6., 2., 8., 4.]])

In [5]:
y = np.max(x_col, axis=1)
y

array([8., 7., 8., 9., 9., 9., 9., 9., 8., 4., 7., 9., 4., 7., 7., 9., 9.,
       8., 9., 7., 7., 9., 9., 8.])

In [6]:
y = y.reshape(2, 2, 2, 3).transpose(0,3,1,2)
y

array([[[[8., 9.],
         [9., 4.]],

        [[7., 9.],
         [9., 7.]],

        [[8., 9.],
         [8., 9.]]],


       [[[4., 9.],
         [9., 9.]],

        [[7., 9.],
         [7., 9.]],

        [[7., 8.],
         [7., 8.]]]])

In [7]:
max_index = np.argmax(x_col, axis=1)
max_index

array([1, 1, 0, 0, 1, 1, 1, 0, 3, 1, 0, 0, 3, 0, 0, 0, 1, 2, 3, 1, 3, 0,
       1, 2])

## 3.2 逆伝播

In [8]:
dy = np.ones(y.shape).transpose(0,2,3,1)
dy

array([[[[1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.]]],


       [[[1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.]]]])

In [9]:
dx = np.zeros((2*2, dy.size))
dx

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.]])

In [10]:
dx[max_index.reshape(-1), np.arange(dy.size)] = dy.reshape(-1)
dx

array([[0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 1., 1., 0., 1., 1., 1.,
        0., 0., 0., 0., 0., 1., 0., 0.],
       [1., 1., 0., 0., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 1., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 1., 0., 1., 0., 0., 0.]])

In [11]:
dx = dx.reshape(2, 2, 2, 2, 2, 3).transpose(5,0,1,2,3,4).reshape(3*2*2, 2*2*2)
dx

array([[0., 1., 0., 0., 0., 1., 0., 1.],
       [1., 0., 1., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 1., 0.],
       [0., 0., 1., 1., 1., 0., 0., 0.],
       [1., 1., 0., 0., 0., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 1., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 1.],
       [0., 0., 1., 0., 0., 0., 1., 0.]])

In [12]:
col2im(dx, x.shape, 2, 2, 2, 0)

array([[[[0., 1., 1., 0.],
         [0., 0., 0., 0.],
         [0., 1., 0., 1.],
         [0., 0., 0., 0.]],

        [[0., 1., 0., 1.],
         [0., 0., 0., 0.],
         [1., 0., 1., 0.],
         [0., 0., 0., 0.]],

        [[1., 0., 0., 1.],
         [0., 0., 0., 0.],
         [0., 0., 1., 0.],
         [0., 1., 0., 0.]]],


       [[[0., 0., 1., 0.],
         [0., 1., 0., 0.],
         [0., 0., 1., 0.],
         [0., 1., 0., 0.]],

        [[1., 0., 0., 1.],
         [0., 0., 0., 0.],
         [0., 1., 0., 1.],
         [0., 0., 0., 0.]],

        [[1., 0., 0., 0.],
         [0., 0., 1., 0.],
         [0., 0., 0., 0.],
         [0., 1., 1., 0.]]]])

# 4 Pooling層

In [13]:
class Pooling:
    
    def __init__(self, pool):
        self.pool = pool
        
    def forward(self, x):
        self.xshape = x.shape
        self.x_b, self.x_c, self.x_h, self.x_w = x.shape
        self.y_h = self.x_h//self.pool if self.x_h%self.pool==0 else self.x_h//self.pool+1
        self.y_w = self.x_w//self.pool if self.x_w%self.pool==0 else self.x_w//self.pool+1
        
        x_col = im2col(x, self.pool, self.y_h, self.pool, 0).T.reshape(-1,self.pool*self.pool)
        y = np.max(x_col, axis=1)
        self.y = y.reshape(self.x_b, self.y_h, self.y_w, self.x_c).transpose(0,3,1,2)
        self.max_index = np.argmax(x_col, axis=1)
        return self.y
    
    def backward(self, dy):
        dy = dy.transpose(0,2,3,1)
        dx = np.zeros((self.pool*self.pool, dy.size))
        dx[self.max_index.reshape(-1), np.arange(dy.size)] = dy.reshape(-1)
        dx = dx.reshape(self.pool, self.pool, self.x_b, self.y_h, self.y_w, self.x_c)
        dx = dx.transpose(5,0,1,2,3,4)
        dx = dx.reshape(self.x_c*self.pool*self.pool, self.x_b*self.y_h*self.y_w)
        self.dx = col2im(dx, self.xshape, self.pool, self.y_h, self.pool, 0)
        return self.dx

In [14]:
pool = Pooling(2)

In [15]:
x.shape

(2, 3, 4, 4)

In [16]:
y = pool.forward(x)
y

array([[[[8., 9.],
         [9., 4.]],

        [[7., 9.],
         [9., 7.]],

        [[8., 9.],
         [8., 9.]]],


       [[[4., 9.],
         [9., 9.]],

        [[7., 9.],
         [7., 9.]],

        [[7., 8.],
         [7., 8.]]]])